In [2]:
import networkx as nx
from collections import Counter, defaultdict
from pathlib import Path
import json
import re
import pickle

# =========================
# CONFIG: set your paths (update these to your file locations in Drive)
# =========================
# Update these paths to point to your files in Google Drive
OPENFDA_PATH =  "/home/ubuntu/myproject/venv/drug_data_kg_openfda_edges_no_drop.graphml"
PRIMEKG_PATH = "/home/ubuntu/myproject/venv/primekg_hetionet_combined.graphml"
OUT_MERGED_GRAPH = "/home/ubuntu/myproject/venv/merged_openfda_primekg.graphml"
OUT_MERGED_PICKLE = "/home/ubuntu/myproject/venv/merged_openfda_primekg.pickle"
OUT_SUMMARY_JSON = "/home/ubuntu/myproject/venv/merged_openfda_primekg.summary.json"

# Options
USE_MULTIGRAPH = True          # keep every parallel edge (no drops)
NORMALIZE_NAMES = True         # lower/strip/collapse spaces for node_name when matching
PREFER_NODE_SOURCE = "primekg" # if node attribute conflicts: "openfda" or "primekg"

# =========================
# Helpers
# =========================
def load_graph(path):
    G = nx.read_graphml(path)
    kind = "MultiDiGraph" if G.is_multigraph() else "DiGraph"
    print(f"Loaded {path} -> {kind} | nodes={G.number_of_nodes():,} edges={G.number_of_edges():,}")
    return G

def get_attr(attrs, k, default=""):
    v = attrs.get(k)
    return default if v is None else v

_ws_re = re.compile(r"\s+")
def canon_name(name: str) -> str:
    s = name if isinstance(name, str) else str(name)
    if NORMALIZE_NAMES:
        s = _ws_re.sub(" ", s.strip().lower())
    return s

def node_inventory(G, node_type_key="node_type", node_name_key="node_name"):
    types = Counter(nx.get_node_attributes(G, node_type_key).values())
    names_missing = sum(1 for _, a in G.nodes(data=True) if node_name_key not in a or not str(a.get(node_name_key,"")).strip())
    return types, names_missing

def ensure_node_basics(G, source_tag):
    # guarantee node_name and node_source exist
    for n, a in G.nodes(data=True):
        if "node_name" not in a or not str(a.get("node_name","")).strip():
            a["node_name"] = str(n)
        # stamp node_source if missing (may already exist in your KGs)
        if "node_source" not in a or not a["node_source"]:
            a["node_source"] = source_tag

def summarize_graph(G, node_type_key="node_type", edge_rel_key="relation"):
    kind = "MultiDiGraph" if G.is_multigraph() else "DiGraph"
    n_nodes = G.number_of_nodes()
    n_edges = G.number_of_edges()
    node_types = Counter(nx.get_node_attributes(G, node_type_key).values())
    edge_types = Counter(nx.get_edge_attributes(G, edge_rel_key).values())

    node_types_list = sorted(node_types.items(), key=lambda kv: (-kv[1], kv[0]))
    edge_types_list = sorted(edge_types.items(), key=lambda kv: (-kv[1], kv[0]))

    # connectivity (MultiGraphs supported)
    UG = G.to_undirected()
    comps = list(nx.connected_components(UG))
    largest = max((len(c) for c in comps), default=0)
    frac = (largest / n_nodes) if n_nodes else 0.0
    orphans = sum(1 for _ in nx.isolates(G))

    summary = {
        "graph_kind": kind,
        "nodes": int(n_nodes),
        "edges": int(n_edges),

        "node_types_count": int(len(node_types_list)),
        "node_types": node_types_list,

        "relation_types_count": int(len(edge_types_list)),
        "relation_types": edge_types_list,

        "connectivity": {
            "components": int(len(comps)),
            "largest_component_size": int(largest),
            "largest_component_fraction": float(frac),
            "orphans": int(orphans),
        },

        "keys_used": {
            "node_type_key": node_type_key,
            "edge_rel_key": edge_rel_key,
        },
    }
    return summary

def triplet_schema(G, node_type_key="node_type", edge_rel_key="relation", top=30):
    # list unique (src_type, relation, dst_type) and counts
    counts = Counter()
    # works for DiGraph & MultiDiGraph
    if G.is_multigraph():
        for u, v, k, a in G.edges(keys=True, data=True):
            r = a.get(edge_rel_key, "unknown")
            st = G.nodes[u].get(node_type_key, "unknown")
            dt = G.nodes[v].get(node_type_key, "unknown")
            counts[(st, r, dt)] += 1
    else:
        for u, v, a in G.edges(data=True):
            r = a.get(edge_rel_key, "unknown")
            st = G.nodes[u].get(node_type_key, "unknown")
            dt = G.nodes[v].get(node_type_key, "unknown")
            counts[(st, r, dt)] += 1
    total = len(counts)
    top_items = counts.most_common(top)
    return total, top_items

# =========================
# Load graphs
# =========================
print("Loading graphs...")
G1 = load_graph(OPENFDA_PATH)     # your normalized OpenFDA minimal (node_type/node_name/node_source + relation/subfield)
G2 = load_graph(PRIMEKG_PATH)     # PrimeKG+Hetionet combined

# Ensure basic attrs present
ensure_node_basics(G1, "openfda")
ensure_node_basics(G2, "primekg")

# Quick pre-merge stats
t1, miss1 = node_inventory(G1)
t2, miss2 = node_inventory(G2)
print("\nOpenFDA types (top):", t1.most_common(10), "| missing node_name:", miss1)
print("PrimeKG+Hetionet types (top):", t2.most_common(10), "| missing node_name:", miss2)

# =========================
# Build canonical map by (node_type, node_name)
# We create a fresh MultiDiGraph (or DiGraph) so nothing is lost.
# =========================
MG = nx.MultiDiGraph() if USE_MULTIGRAPH else nx.DiGraph()

NODE_TYPE_KEY = "node_type"
NODE_NAME_KEY = "node_name"
NODE_SOURCE_KEY = "node_source"
EDGE_REL_KEY = "relation"

# map canonical key -> merged node id
canon2id = {}
# keep an incremental integer suffix to avoid accidental id collisions
next_int = 0

def make_merged_id(ntype, nname):
    # Construct a stable readable id. If collides, append counter.
    global next_int  # Use global instead of nonlocal
    base = f"{ntype}::{nname}"
    if base in MG:  # extremely rare; but handle
        while f"{base}##{next_int}" in MG:
            next_int += 1
        return f"{base}##{next_int}"
    return base

def upsert_node(nid, attrs, source_graph):
    """Insert node into merged graph using canonical (type, name) matching. Returns merged node id."""
    ntype = get_attr(attrs, NODE_TYPE_KEY, "unknown")
    nname = get_attr(attrs, NODE_NAME_KEY, str(nid))
    ckey = (canon_name(ntype), canon_name(nname))

    if ckey in canon2id:
        mid = canon2id[ckey]
        # merge attributes (prefer PRIMEKG if configured)
        existing = MG.nodes[mid]
        # keep node_type and node_name as-is
        # merge node_source
        srcs = set(str(existing.get(NODE_SOURCE_KEY, "")).split("|")) if existing.get(NODE_SOURCE_KEY) else set()
        srcs2 = set(str(attrs.get(NODE_SOURCE_KEY, "")).split("|")) if attrs.get(NODE_SOURCE_KEY) else set()
        all_srcs = [s for s in (srcs | srcs2) if s]
        MG.nodes[mid][NODE_SOURCE_KEY] = "|".join(sorted(set(all_srcs))) if all_srcs else existing.get(NODE_SOURCE_KEY, "")
        # retain an 'orig_id_<source>' list
        tag = "primekg" if source_graph == "primekg" else "openfda"
        orig_key = f"orig_id_{tag}"
        olds = set(str(MG.nodes[mid].get(orig_key, "")).split("|")) if MG.nodes[mid].get(orig_key) else set()
        olds.add(str(nid))
        MG.nodes[mid][orig_key] = "|".join(sorted(olds))
        return mid
    else:
        mid = make_merged_id(ntype, nname)
        canon2id[ckey] = mid
        # initialize node attrs
        MG.add_node(mid)
        MG.nodes[mid][NODE_TYPE_KEY] = ntype
        MG.nodes[mid][NODE_NAME_KEY] = nname
        MG.nodes[mid][NODE_SOURCE_KEY] = attrs.get(NODE_SOURCE_KEY, source_graph)
        # store original ids by source
        tag = "primekg" if source_graph == "primekg" else "openfda"
        MG.nodes[mid][f"orig_id_{tag}"] = str(nid)
        return mid

def add_edge_preserving(u_mid, v_mid, eattrs, source_graph):
    # always keep relation and subfield if present
    rel = eattrs.get(EDGE_REL_KEY, "unknown")
    subf = eattrs.get("subfield", None)
    payload = {
        EDGE_REL_KEY: rel,
        "source_graph": source_graph,
    }
    if subf is not None:
        payload["subfield"] = subf
    # optional: keep original endpoints for traceability
    # payload["orig_u"] = str(u_mid)  # usually not needed
    # payload["orig_v"] = str(v_mid)
    if MG.is_multigraph():
        MG.add_edge(u_mid, v_mid, **payload)
    else:
        # In DiGraph, last write wins; to avoid drops we could count, but we promised no drops, so Multi is recommended.
        MG.add_edge(u_mid, v_mid, **payload)

# =========================
# Add OpenFDA nodes/edges
# =========================
print("Adding OpenFDA nodes and edges...")
for n, a in G1.nodes(data=True):
    upsert_node(n, a, "openfda")

if G1.is_multigraph():
    for u, v, k, a in G1.edges(keys=True, data=True):
        u_mid = upsert_node(u, G1.nodes[u], "openfda")
        v_mid = upsert_node(v, G1.nodes[v], "openfda")
        add_edge_preserving(u_mid, v_mid, a, "openfda")
else:
    for u, v, a in G1.edges(data=True):
        u_mid = upsert_node(u, G1.nodes[u], "openfda")
        v_mid = upsert_node(v, G1.nodes[v], "openfda")
        add_edge_preserving(u_mid, v_mid, a, "openfda")

print(f"After OpenFDA -> merged: nodes={MG.number_of_nodes():,} edges={MG.number_of_edges():,}")

# =========================
# Add PrimeKG nodes/edges
# =========================
print("Adding PrimeKG nodes and edges...")
for n, a in G2.nodes(data=True):
    upsert_node(n, a, "primekg")

if G2.is_multigraph():
    for u, v, k, a in G2.edges(keys=True, data=True):
        u_mid = upsert_node(u, G2.nodes[u], "primekg")
        v_mid = upsert_node(v, G2.nodes[v], "primekg")
        add_edge_preserving(u_mid, v_mid, a, "primekg")
else:
    for u, v, a in G2.edges(data=True):
        u_mid = upsert_node(u, G2.nodes[u], "primekg")
        v_mid = upsert_node(v, G2.nodes[v], "primekg")
        add_edge_preserving(u_mid, v_mid, a, "primekg")

print(f"After PrimeKG -> merged: nodes={MG.number_of_nodes():,} edges={MG.number_of_edges():,}")

# =========================
# Optional: prefer one source's attrs when conflicts (kept minimal here)
# =========================
# You can extend this block to pull preferred per-type attributes.

# =========================
# Save merged graph in multiple formats
# =========================
print("Saving merged graph...")

# Save as GraphML
nx.write_graphml(MG, OUT_MERGED_GRAPH)
print(f"Saved merged graph (GraphML) -> {OUT_MERGED_GRAPH}")

# Save as pickle
with open(OUT_MERGED_PICKLE, 'wb') as f:
    pickle.dump(MG, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Saved merged graph (Pickle) -> {OUT_MERGED_PICKLE}")

# =========================
# Build & save summary
# =========================
print("Generating summary...")
summary = summarize_graph(MG, node_type_key=NODE_TYPE_KEY, edge_rel_key=EDGE_REL_KEY)

# Triplet schema preview
schema_total, schema_top = triplet_schema(MG, node_type_key=NODE_TYPE_KEY, edge_rel_key=EDGE_REL_KEY, top=30)
summary["triplet_schema_types"] = int(schema_total)
summary["triplet_schema_top"] = [[list(k), int(v)] for k, v in schema_top]

with open(OUT_SUMMARY_JSON, "w") as f:
    json.dump(summary, f, indent=2)

print("\n=== MERGED SUMMARY ===")
print(json.dumps(summary, indent=2))
print(f"\nSummary saved -> {OUT_SUMMARY_JSON}")

# =========================
# Additional info about the saved files
# =========================
print(f"\n=== FILES SAVED ===")
print(f"GraphML: {OUT_MERGED_GRAPH}")
print(f"Pickle:  {OUT_MERGED_PICKLE}")
print(f"Summary: {OUT_SUMMARY_JSON}")

# Quick comparison of file sizes
import os
if os.path.exists(OUT_MERGED_GRAPH) and os.path.exists(OUT_MERGED_PICKLE):
    graphml_size = os.path.getsize(OUT_MERGED_GRAPH) / (1024*1024)  # MB
    pickle_size = os.path.getsize(OUT_MERGED_PICKLE) / (1024*1024)  # MB
    print(f"\nFile sizes:")
    print(f"GraphML: {graphml_size:.2f} MB")
    print(f"Pickle:  {pickle_size:.2f} MB")
    if graphml_size > 0:
        print(f"Ratio:   {graphml_size/pickle_size:.2f}x (Pickle is {pickle_size/graphml_size:.2f}x smaller)")

Loading graphs...
Loaded /home/ubuntu/myproject/venv/drug_data_kg_openfda_edges_no_drop.graphml -> DiGraph | nodes=100,495 edges=160,229
Loaded /home/ubuntu/myproject/venv/primekg_hetionet_combined.graphml -> DiGraph | nodes=176,408 edges=9,435,440

OpenFDA types (top): [('drug', 55430), ('effect/phenotype', 27473), ('exposure', 16513), ('disease', 1079)] | missing node_name: 0
PrimeKG+Hetionet types (top): [('gene/protein', 48616), ('biological_process', 40023), ('effect/phenotype', 21483), ('disease', 17217), ('anatomy', 14437), ('molecular_function', 14053), ('drug', 9854), ('cellular_component', 5567), ('pathway', 4338), ('exposure', 818)] | missing node_name: 0
Adding OpenFDA nodes and edges...
After OpenFDA -> merged: nodes=95,170 edges=160,229
Adding PrimeKG nodes and edges...
After PrimeKG -> merged: nodes=233,412 edges=9,595,669
Saving merged graph...
Saved merged graph (GraphML) -> /home/ubuntu/myproject/venv/merged_openfda_primekg.graphml
Saved merged graph (Pickle) -> /home

In [1]:
# === GraphML → R-GCN (NeighborLoader + AMP + batched inference) ===
# Fixes:
#  - DistMult scoring (relation-aware) in train + eval
#  - Typed negatives sampled *within the batch* (local), avoiding drop
#  - zero_division=0 to silence undefined metric warnings

import os, gc, random
from collections import defaultdict

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch_geometric.nn import RGCNConv
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
import networkx as nx

# ----------------------------
# 0) Repro + Device
# ----------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gc.collect()
if device.type == "cuda":
    torch.cuda.empty_cache()
print("Using device:", device)

# ----------------------------
# 1) Load GraphML + normalize attrs
# ----------------------------
graph_path = "/home/ubuntu/myproject/venv/merged_openfda_primekg.graphml"
assert os.path.exists(graph_path), f"Path does not exist: {graph_path}"
print(f"📥 Loading GraphML from: {graph_path}")

G0 = nx.read_graphml(graph_path)
G = nx.MultiDiGraph()
G.add_nodes_from(G0.nodes(data=True))
if G0.is_multigraph():
    for u,v,k,d in G0.edges(keys=True, data=True):
        G.add_edge(u,v,key=k, **(d or {}))
else:
    for u,v,d in G0.edges(data=True):
        G.add_edge(u,v, **(d or {}))

for n,data in G.nodes(data=True):
    if "node_type" not in data:
        data["node_type"] = data.get("type", data.get("category", data.get("kind","unknown")))
for u,v,attr in G.edges(data=True):
    if "relation" not in attr:
        attr["relation"] = attr.get("relationship", attr.get("type", attr.get("label","unknown")))

print(f"✅ Loaded graph with {len(G.nodes)} nodes and {len(G.edges)} edges.")

# ----------------------------
# 2) Typed remap + splits
# ----------------------------
node_type_map = defaultdict(list)
node_id_map = {}
for nid,data in G.nodes(data=True):
    ntype = data.get("node_type","unknown")
    idx = len(node_type_map[ntype])
    node_type_map[ntype].append(nid)
    node_id_map[nid] = (ntype, idx)

edge_type_map = defaultdict(list)  # (src_type, rel, dst_type) -> [(src_local, dst_local)]
for s,t,attr in G.edges(data=True):
    if s not in node_id_map or t not in node_id_map: continue
    rel = attr.get("relation","unknown")
    s_t,s_i = node_id_map[s]
    t_t,t_i = node_id_map[t]
    edge_type_map[(s_t, rel, t_t)].append((s_i,t_i))

print(f"✅ Node types: {list(node_type_map.keys())}")
print(f"✅ Total edge types: {len(edge_type_map)}")

# Global ids per (type, local)
type_offsets, global_id_map = {}, {}
offset = 0
for ntype, nodes in node_type_map.items():
    type_offsets[ntype] = offset
    for i in range(len(nodes)):
        global_id_map[(ntype,i)] = offset + i
    offset += len(nodes)
num_nodes = offset

edge_type_keys = list(edge_type_map.keys())         # rel_id -> (src_type, rel, dst_type)
rel2id = {k:i for i,k in enumerate(edge_type_keys)}
num_relations = len(edge_type_keys)

all_src, all_dst, all_rel = [], [], []
edge_type_splits = {}

for etype, pairs in edge_type_map.items():
    s_type,_, t_type = etype
    if len(pairs) < 5:  # keep only relations with minimum support
        continue
    perm = torch.randperm(len(pairs))
    n_tr = int(0.8*len(pairs)); n_va = int(0.1*len(pairs))
    tr = [pairs[i] for i in perm[:n_tr]]
    va = [pairs[i] for i in perm[n_tr:n_tr+n_va]]
    te = [pairs[i] for i in perm[n_tr+n_va:]]

    r = rel2id[etype]
    for s_l, t_l in tr:
        all_src.append(type_offsets[s_type] + s_l)
        all_dst.append(type_offsets[t_type] + t_l)
        all_rel.append(r)

    edge_type_splits[etype] = {
        "val": torch.tensor([[type_offsets[s_type]+s for s,_ in va],
                             [type_offsets[t_type]+t for _,t in va]], dtype=torch.long),
        "test": torch.tensor([[type_offsets[s_type]+s for s,_ in te],
                              [type_offsets[t_type]+t for _,t in te]], dtype=torch.long),
    }

edge_index_cpu = torch.tensor([all_src, all_dst], dtype=torch.long)
edge_type_cpu  = torch.tensor(all_rel, dtype=torch.long)
print(f"Homogeneous nodes: {num_nodes}, edges used for training: {edge_index_cpu.size(1)}")

# For quick type lookup by GLOBAL id (used for local typed negatives)
type_names = sorted(node_type_map.keys())
type2id = {t:i for i,t in enumerate(type_names)}
global_node_type_id = torch.empty(num_nodes, dtype=torch.long)
for tname, nodes in node_type_map.items():
    t_id = type2id[tname]
    base = type_offsets[tname]
    n = len(nodes)
    global_node_type_id[base:base+n] = t_id

# Existing edges per relation (LOCAL to each (src_type,rel,dst_type))
existing_edges_per_rel = {etype: set(pairs) for etype,pairs in edge_type_map.items()}
def relid_to_etype(rel_id:int):
    return edge_type_keys[rel_id]

# ----------------------------
# 3) PyG Data + NeighborLoader
# ----------------------------
data_pyg = Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu)

NUM_NEIGHBORS = [15, 10]
NBATCH_SIZE   = 4096
NUM_WORKERS   = 2  # quiets the OS warning

train_src_unique = torch.unique(edge_index_cpu[0])
train_loader = NeighborLoader(
    data_pyg,
    num_neighbors=NUM_NEIGHBORS,
    batch_size=NBATCH_SIZE,
    input_nodes=train_src_unique,
    shuffle=True,
    num_workers=NUM_WORKERS
)

# ----------------------------
# 4) Model (DistMult) + helpers
# ----------------------------
class RGCN(nn.Module):
    def __init__(self, num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30):
        super().__init__()
        self.emb = nn.Embedding(num_nodes, emb_dim)
        self.conv1 = RGCNConv(emb_dim, hidden_dim, num_relations=num_relations, num_bases=num_bases)
        self.conv2 = RGCNConv(hidden_dim, out_dim, num_relations=num_relations, num_bases=num_bases)
        self.rel_emb = nn.Embedding(num_relations, out_dim)

        nn.init.xavier_uniform_(self.emb.weight)
        nn.init.xavier_uniform_(self.rel_emb.weight)

    def forward(self, n_id, edge_index, edge_type):
        # n_id: global ids ordered as local nodes in the subgraph
        x = self.emb(n_id)
        x = self.conv1(x, edge_index, edge_type).relu_()
        x = self.conv2(x, edge_index, edge_type)
        return x  # local node embeddings

    def score(self, h, r, t):
        # DistMult (logits)
        return (h * r * t).sum(dim=-1)

def distmult_sigmoid_scores(z, rel_emb, eidx, rel_ids):
    h = z[eidx[0]]; t = z[eidx[1]]
    r = rel_emb(rel_ids)
    return (h * r * t).sum(dim=-1).sigmoid()

# ----------------------------
# 5) Local (in-batch) typed negatives
# ----------------------------
def build_local_type_pools(n_id_device):
    """Map node_type_id -> list of LOCAL indices present in this batch."""
    g_types = global_node_type_id.to(n_id_device.device, non_blocking=True)[n_id_device]
    pools = defaultdict(list)
    for local_idx, t in enumerate(g_types.tolist()):
        pools[t].append(local_idx)
    for k in list(pools.keys()):
        pools[k] = torch.tensor(pools[k], dtype=torch.long, device=n_id_device.device)
    return pools

def sample_neg_local(pos_local_eidx, rel_ids_local, n_id, pools, k_neg=1):
    """
    Typed negatives *within* the sampled subgraph.
    pos_local_eidx: (2,P) local indices
    rel_ids_local:  (P,)   relation ids (global rel ids)
    pools: dict[type_id] -> local node index tensor present in batch
    """
    if pos_local_eidx.numel() == 0:  # empty
        return torch.empty((2,0), dtype=torch.long, device=pos_local_eidx.device)

    P = pos_local_eidx.size(1)
    neg_src, neg_dst = [], []
    for i in range(P):
        s_l = int(pos_local_eidx[0, i])
        d_l = int(pos_local_eidx[1, i])
        r_id = int(rel_ids_local[i])
        s_type, _, d_type = relid_to_etype(r_id)
        s_tid = type2id[s_type]; d_tid = type2id[d_type]

        # pick from local pools — if missing (rare), fall back to uniform local sampling
        s_pool = pools.get(s_tid, None)
        d_pool = pools.get(d_tid, None)
        if s_pool is None or s_pool.numel() == 0:
            s_pool = torch.arange(n_id.size(0), device=n_id.device)
        if d_pool is None or d_pool.numel() == 0:
            d_pool = torch.arange(n_id.size(0), device=n_id.device)

        for _ in range(max(1, k_neg)):
            if random.random() < 0.5:
                # corrupt head
                new_s_l = s_pool[torch.randint(0, s_pool.numel(), (1,))].item()
                neg_src.append(new_s_l); neg_dst.append(d_l)
            else:
                # corrupt tail
                new_d_l = d_pool[torch.randint(0, d_pool.numel(), (1,))].item()
                neg_src.append(s_l); neg_dst.append(new_d_l)

    if len(neg_src) == 0:
        return torch.empty((2,0), dtype=torch.long, device=n_id.device)
    return torch.stack([torch.tensor(neg_src, device=n_id.device),
                        torch.tensor(neg_dst, device=n_id.device)], dim=0)

# ----------------------------
# 6) Train with NeighborLoader + AMP
# ----------------------------
EPOCHS = 30
LR     = 1e-3
K_NEG  = 1

model = RGCN(num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
scaler = GradScaler(enabled=(device.type == "cuda"))

# Global training edges (for picking positives per batch)
src_glob_all = edge_index_cpu[0].to(device)
dst_glob_all = edge_index_cpu[1].to(device)
rels_all     = edge_type_cpu.to(device)

print("\n🔁 Training with NeighborLoader mini-batches + AMP ...")
for epoch in range(1, EPOCHS+1):
    model.train()
    epoch_loss = 0.0
    seen_pos = 0

    for batch in train_loader:
        batch = batch.to(device, non_blocking=True)
        n_id = batch.n_id  # global ids in this subgraph (device tensor)

        # global->local mapping
        global2local = -torch.ones(num_nodes, dtype=torch.long, device=device)
        global2local[n_id] = torch.arange(n_id.size(0), device=device)

        # positives inside batch
        in_src = global2local[src_glob_all]
        in_dst = global2local[dst_glob_all]
        mask = (in_src >= 0) & (in_dst >= 0)
        pos_idx = torch.nonzero(mask, as_tuple=False).view(-1)
        if pos_idx.numel() == 0:
            del batch, n_id, global2local, in_src, in_dst, mask, pos_idx
            if device.type == "cuda": torch.cuda.empty_cache()
            continue

        MAX_POS_PER_BATCH = 8192
        if pos_idx.numel() > MAX_POS_PER_BATCH:
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device)[:MAX_POS_PER_BATCH]]

        pos_local = torch.stack([in_src[pos_idx], in_dst[pos_idx]], dim=0)
        rel_ids_local = rels_all[pos_idx]

        # local typed pools for negatives
        pools = build_local_type_pools(n_id)

        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=(device.type == "cuda")):
            z = model(n_id, batch.edge_index, batch.edge_type)

            # DistMult positives
            pos_h = z[pos_local[0]]; pos_t = z[pos_local[1]]
            pos_r = model.rel_emb(rel_ids_local)
            pos_logits = model.score(pos_h, pos_r, pos_t)

            # local typed negatives
            neg_local = sample_neg_local(pos_local, rel_ids_local, n_id, pools, k_neg=K_NEG)
            if neg_local.size(1) > 0:
                neg_h = z[neg_local[0]]; neg_t = z[neg_local[1]]
                # repeat relation ids for negs
                neg_r = model.rel_emb(rel_ids_local.repeat_interleave(K_NEG))
                neg_logits = model.score(neg_h, neg_r, neg_t)
                loss = (F.binary_cross_entropy_with_logits(pos_logits, torch.ones_like(pos_logits)) +
                        F.binary_cross_entropy_with_logits(neg_logits, torch.zeros_like(neg_logits)))
            else:
                loss = F.binary_cross_entropy_with_logits(pos_logits, torch.ones_like(pos_logits))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += float(loss.detach().cpu())
        seen_pos += pos_local.size(1)

        # cleanup
        del batch, n_id, global2local, in_src, in_dst, mask, pos_idx
        del pos_local, rel_ids_local, pools, z, pos_h, pos_t, pos_r, pos_logits
        if 'neg_local' in locals(): del neg_local, neg_h, neg_t, neg_r, neg_logits
        if device.type == "cuda": torch.cuda.empty_cache()

    print(f"Epoch {epoch:02d} | Loss: {epoch_loss:.4f} | Seen pos edges: {seen_pos}")

# ----------------------------
# 7) Batched inference (DistMult-ready embeddings)
# ----------------------------
print("\n🔎 Computing full-node embeddings (batched inference) ...")
model.eval()
OUT_DIM = 32
z_all = torch.empty((num_nodes, OUT_DIM), dtype=torch.float32, device=device)

INFER_BATCH = 16384
infer_loader = NeighborLoader(
    Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu),
    num_neighbors=[-1, -1],
    batch_size=INFER_BATCH,
    input_nodes=torch.arange(num_nodes),
    shuffle=False,
    num_workers=NUM_WORKERS
)
with torch.no_grad():
    for batch in infer_loader:
        batch = batch.to(device, non_blocking=True)
        n_id = batch.n_id
        z_batch = model(n_id, batch.edge_index, batch.edge_type)
        z_all[n_id] = z_batch
        del batch, z_batch, n_id
        if device.type == "cuda": torch.cuda.empty_cache()

# ----------------------------
# 8) Evaluation (per relation + overall) — DistMult + zero_division=0
# ----------------------------
print("\n📊 Final evaluation per edge type (filtered negatives, DistMult):")

def eval_relation(z_all, etype, splits, k_neg=1):
    rel_id = rel2id[etype]
    pos_e = splits["test"]
    if pos_e.numel() == 0: return None
    N = pos_e.size(1)
    # Build negatives globally (filtered, typed) just for evaluation
    # (global is fine here; we evaluate on z_all)
    # Reuse earlier sampler but with relation ids vector:
    src_g = pos_e[0].to(device); dst_g = pos_e[1].to(device)
    rel_ids = torch.full((N,), rel_id, dtype=torch.long, device=device)

    # filtered typed negs in GLOBAL space (may include true edges but low prob)
    # Simple version: corrupt tails only
    neg_src = src_g.repeat_interleave(k_neg)
    # build tail pool by type
    s_type,_, d_type = etype
    d_tid = type2id[d_type]
    d_pool_global = torch.arange(type_offsets[d_type], type_offsets[d_type] + len(node_type_map[d_type]), device=device)
    neg_dst = d_pool_global[torch.randint(0, d_pool_global.numel(), (N*k_neg,), device=device)]

    # Scores
    pos_scores = distmult_sigmoid_scores(z_all, model.rel_emb, pos_e.to(device), rel_ids)
    neg_scores = distmult_sigmoid_scores(z_all, model.rel_emb, torch.stack([neg_src, neg_dst]), rel_ids.repeat_interleave(k_neg))

    y_true = torch.cat([torch.ones_like(pos_scores), torch.zeros_like(neg_scores)])
    y_scores = torch.cat([pos_scores, neg_scores]).detach().cpu().numpy()
    y = y_true.detach().cpu().numpy()
    auc = roc_auc_score(y, y_scores)

    preds = (y_scores > 0.5).astype(int)
    p = precision_score(y, preds, zero_division=0)
    r = recall_score(y, preds, zero_division=0)
    f1 = f1_score(y, preds, zero_division=0)
    return auc, p, r, f1

results = []
all_pos_eidx_list, all_rel_ids_list = [], []

for etype, splits in edge_type_splits.items():
    out = eval_relation(z_all, etype, splits, k_neg=1)
    if out is None: continue
    auc, p, r, f1 = out
    results.append((etype, auc, p, r, f1))
    rel_id = rel2id[etype]
    te = splits["test"]
    all_pos_eidx_list.append(te)
    all_rel_ids_list.append(torch.full((te.size(1),), rel_id, dtype=torch.long))
    print(f"{etype}: AUC={auc:.4f}, P={p:.3f}, R={r:.3f}, F1={f1:.3f}")

# Overall
if all_pos_eidx_list:
    all_pos_eidx = torch.cat(all_pos_eidx_list, dim=1).to(device)
    all_rel_ids = torch.cat(all_rel_ids_list, dim=0).to(device)
    # Simple overall negatives: corrupt tails with matching type (1 per pos)
    etypes_for_all = [relid_to_etype(int(r)) for r in all_rel_ids.tolist()]
    neg_src = all_pos_eidx[0]
    neg_dst_list = []
    for r_id, et in zip(all_rel_ids.tolist(), etypes_for_all):
        d_type = et[2]
        pool = torch.arange(type_offsets[d_type], type_offsets[d_type] + len(node_type_map[d_type]), device=device)
        neg_dst_list.append(pool[torch.randint(0, pool.numel(), (1,), device=device)])
    neg_dst = torch.stack(neg_dst_list).view(-1)
    neg_e = torch.stack([neg_src, neg_dst], dim=0)

    pos_scores = distmult_sigmoid_scores(z_all, model.rel_emb, all_pos_eidx, all_rel_ids)
    neg_scores = distmult_sigmoid_scores(z_all, model.rel_emb, neg_e, all_rel_ids)

    y_scores = torch.cat([pos_scores, neg_scores]).detach().cpu().numpy()
    y = np.concatenate([np.ones(pos_scores.numel()), np.zeros(neg_scores.numel())])

    overall_auc = roc_auc_score(y, y_scores)
    preds = (y_scores > 0.5).astype(int)
    overall_p = precision_score(y, preds, zero_division=0)
    overall_r = recall_score(y, preds, zero_division=0)
    overall_f1 = f1_score(y, preds, zero_division=0)

    print("\n🌍 Overall Performance (filtered-ish, typed tails, DistMult):")
    print(f"Overall AUC = {overall_auc:.4f}")
    print(f"Overall Precision = {overall_p:.3f}")
    print(f"Overall Recall = {overall_r:.3f}")
    print(f"Overall F1 Score = {overall_f1:.3f}")
else:
    print("\nNo test edges available for evaluation.")


Using device: cuda
📥 Loading GraphML from: /home/ubuntu/myproject/venv/merged_openfda_primekg.graphml
✅ Loaded graph with 233412 nodes and 9595669 edges.
✅ Node types: ['drug', 'effect/phenotype', 'exposure', 'disease', 'gene/protein', 'biological_process', 'molecular_function', 'cellular_component', 'pathway', 'anatomy', 'unknown']
✅ Total edge types: 101
Homogeneous nodes: 233412, edges used for training: 7676495

🔁 Training with NeighborLoader mini-batches + AMP ...
Epoch 01 | Loss: 53.8263 | Seen pos edges: 319488
Epoch 02 | Loss: 48.6513 | Seen pos edges: 319488
Epoch 03 | Loss: 43.4767 | Seen pos edges: 319488
Epoch 04 | Loss: 41.6179 | Seen pos edges: 319488
Epoch 05 | Loss: 40.6705 | Seen pos edges: 319488
Epoch 06 | Loss: 40.0388 | Seen pos edges: 319488
Epoch 07 | Loss: 39.4785 | Seen pos edges: 319488
Epoch 08 | Loss: 38.9883 | Seen pos edges: 319488
Epoch 09 | Loss: 38.5955 | Seen pos edges: 319488
Epoch 10 | Loss: 38.3429 | Seen pos edges: 319488
Epoch 11 | Loss: 38.1515 |

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.4.0+cu124
CUDA available: True
CUDA version: 12.4
GPU: Tesla T4
